<a href="https://colab.research.google.com/github/lenyabloko/SemEval2020/blob/master/Pikes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
sudo apt-get install openjdk-8-jdk
sudo apt-get install maven

wget https://knowledgestore.fbk.eu/files/pikes/download/pikes-all.tar.gz # Download the full package
tar xzf pikes-all.tar.gz
cd pikes
bash run.sh
gcloud compute firewall-rules create pikes --allow tcp:8011 --source-tags=pikes --source-ranges=0.0.0.0/0 --description="pikes instance web server"

In [0]:
#!pip install stanfordnlp

import os
os.environ['PYTHONPATH'] = "/usr/local/bin/python"
!echo $PYTHONPATH
!which python
!python --version

import sys
sys.path.append('/usr/local/lib/python3.6/site-packages/')

import stanfordnlp
stanfordnlp.download('en', force=True)   

nlp = stanfordnlp.Pipeline() # This sets up a default neural pipeline in English
doc = nlp("Barack Obama was born in Hawaii.  He was elected president in 2008.")
doc.sentences[0].print_dependencies()

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

#!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-1/train_delex.jsonl /content
#!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-1/test_delex.jsonl /content

In [3]:
import pandas as pd

prefix = '/content/'
#train_df = pd.read_json(prefix + 'test_delex.jsonl', orient='records', lines=True)
df = pd.read_json(prefix + 'test_delex.jsonl', orient='columns')
df.head()

,claim,evidence,label
0,Unfortunately letting them behave badly becaus...,sentenceID113000,0
1,If ORGANIZATIONc1 ended antiretroviral coverag...,sentenceID113001,0
2,MISCc1 may not have read their bill before the...,sentenceID113002,0
3,People close to Mr PERSONc1 have suggested tha...,sentenceID113003,0
4,This must all happen fast if policies are to t...,sentenceID113004,0


In [32]:
# requests for fetching html of website
import requests

endpoint = 'http://35.245.146.92:8011/text2naf?text='

# Make the request to a url
r = requests.get(endpoint)
c = r.content
print(c)

b'<?xml version="1.0" encoding="UTF-8"?>\n<NAF xml:lang="en" version="v3">\n  <nafHeader>\n    <fileDesc creationtime="2020-02-24T08:39:46Z" />\n    <public uri="http://pikes.fbk.eu/" />\n    <linguisticProcessors layer="text">\n      <lp name="Stanford CoreNLP" beginTimestamp="2020-02-24T08:39:46.474+0000" endTimestamp="2020-02-24T08:39:46.774+0000" />\n    </linguisticProcessors>\n    <linguisticProcessors layer="naf-filter">\n      <lp name="NAF filter" beginTimestamp="2020-02-24T08:39:46.775+0000" endTimestamp="2020-02-24T08:39:46.775+0000" />\n    </linguisticProcessors>\n  </nafHeader>\n  <timeExpressions>\n    <timex3 id="tmx0" type="DATE" value="2020-02-24" />\n  </timeExpressions>\n</NAF>\n'


In [31]:
#!pip install corenlp_xml
!pip install corenlp-xml
from corenlp_xml.document import Document

# requests for fetching html of website
import requests

endpoint = 'http://35.245.146.92:8011/text2naf?text='

for i in range(len(df)-6990) : 
  input_text = df.loc[i, "claim"]
  output_xml = requests.get(endpoint+input_text).content
  print(output_xml)

  d = Document(output_xml)
  print(d.sentences)

  for sentence in d.sentences:
    print(sentence)
    for token in sentence:
      print(token.word)

b'<?xml version="1.0" encoding="UTF-8"?>\n<NAF xml:lang="en" version="v3">\n  <nafHeader>\n    <fileDesc creationtime="2020-02-24T08:33:41Z" />\n    <public uri="http://pikes.fbk.eu/" />\n    <linguisticProcessors layer="text">\n      <lp name="Stanford CoreNLP" beginTimestamp="2020-02-24T08:33:41.125+0000" endTimestamp="2020-02-24T08:33:42.689+0000" />\n    </linguisticProcessors>\n    <linguisticProcessors layer="naf-filter">\n      <lp name="NAF filter" beginTimestamp="2020-02-24T08:33:42.698+0000" endTimestamp="2020-02-24T08:33:42.701+0000" />\n    </linguisticProcessors>\n  </nafHeader>\n  <raw><![CDATA[Unfortunately letting them behave badly because he did create free will and that doesnt mean anything if he distorts our will whenever or for whatever reason had consequences on me]]></raw>\n  <text>\n    <wf id="w1" sent="1" para="1" offset="0" length="13">Unfortunately</wf>\n    <wf id="w2" sent="1" para="1" offset="14" length="7">letting</wf>\n    <wf id="w3" sent="1" para="1" o

In [30]:
!pip install corenlpy
!pip install corenlp-xml-reader

from corenlpy import AnnotatedText as A

# requests for fetching html of website
import requests

endpoint = 'http://35.245.146.92:8011/text2naf?text='

for i in range(len(df)-6990) : 
  input_text = df.loc[i, "claim"]
  output_xml = requests.get(endpoint+input_text).content
  print(output_xml)

  xml = open(output_xml).read()
  annotated_text = A(xml)
  print(annotated_text)

  #Usually you’ll access parts of the document using the sentences list.
  len(annotated_text.sentences)
  sentence = annotated_text.sentences[0]
  print(sentence)
  print(sentence.keys())
  #['tokens', 'entities', 'references', 'mentions', 'root', 'id']

  #A Sentence is a special class that, for the most part, feels like a simple dict.
  #The tokens property is a list of the sentence’s tokens:

  tokens = sentence['tokens'][1]
  print(tokens)
  #' 0: Obama (10,14) NNP PERSON'

  term = sentence['tokens'][7]
  print(term)

  keys = tokens.keys()
  print(keys)
  #['word', 'character_offset_begin', 'character_offset_end', 'pos',


ModuleNotFoundError: ignored